In [7]:
import requests
import pandas as pd
from glob import glob

In [3]:
category_save = "http://127.0.0.1:8081/main/category/save"
request_body = {"catename":"반팔"}
requests.post(url=category_save, json=request_body)

<Response [201]>

* 상의: 1
    * 반팔: 5
    * 긴팔: 6
    * 셔츠: 7

* 바지: 2
    * 반바지: 8
    * 슬랙스: 9
    * 데님팬츠: 10

* 스커트: 3
    * 미니스커트: 11
    * 롱스커트: 12

* 아우터: 4
    * 롱패딩: 13
    * 숏패딩: 14
    * 코트: 15
    * 트렌치 코트: 16

In [50]:
category_list = ['상의', '바지', '스커트', '아우터', '반팔', '긴팔', '셔츠', '반바지', '슬랙스', '데님팬츠', '미니스커트', '롱스커트', '롱패딩', '숏패딩', '코트', '트렌치 코트']

category_save = "http://127.0.0.1:8081/main/category/save"

for category in category_list:
    request_body = {"catename":category}
    requests.post(url=category_save, json=request_body)

In [19]:
csv_file = glob("./상품DataFrame/*.csv")
csv_file

['./상품DataFrame\\바지_데님팬츠.csv',
 './상품DataFrame\\바지_반바지.csv',
 './상품DataFrame\\바지_슬랙스.csv',
 './상품DataFrame\\상의_긴팔.csv',
 './상품DataFrame\\상의_반팔.csv',
 './상품DataFrame\\상의_셔츠.csv',
 './상품DataFrame\\스커트_롱스커트.csv',
 './상품DataFrame\\스커트_미니스커트.csv',
 './상품DataFrame\\아우터_롱패딩.csv',
 './상품DataFrame\\아우터_숏패딩.csv',
 './상품DataFrame\\아우터_코트.csv',
 './상품DataFrame\\아우터_트렌치 코트.csv']

In [20]:
tmp = csv_file[-1].split('\\')[1][:-4]
print(tmp.split('_'))

['아우터', '트렌치 코트']


In [18]:
tmp = csv_file[0].split('\\')[1][:-4]
print(tmp.split('_'))

['바지', '데님팬츠']


In [13]:
df = pd.read_csv(csv_file[0])
df = df.iloc[:, 1:]
df

,상품명,가격,이미지 경로,이미지 개수
0,와이드 데님 팬츠 LIGHT BLUE,"44,100원",['./상품이미지/바지/데님팬츠/와이드 데님 팬츠 LIGHT BLUE1.jpg'],1
1,와이드 데님 팬츠 DEEP GREY,"44,100원",['./상품이미지/바지/데님팬츠/와이드 데님 팬츠 DEEP GREY1.jpg'],1
2,1931 OBJECT JEANS CROP STRAIGHT,"55,380원",['./상품이미지/바지/데님팬츠/1931 OBJECT JEANS CROP STRAI...,1
3,1967 JET BLACK JEANS WIDE STRAIGHT,"63,960원",['./상품이미지/바지/데님팬츠/1967 JET BLACK JEANS WIDE ST...,1
4,8cm 숏버전 추가 0214 Moderation indigo straight fit,"59,000원",['./상품이미지/바지/데님팬츠/8cm 숏버전 추가 0214 Moderation i...,3
...,...,...,...,...
85,레귤러 핏 데님 팬츠 블랙,"39,900원","['./상품이미지/바지/데님팬츠/레귤러 핏 데님 팬츠 블랙1.jpg', './상품이...",3
86,51006 HISHITOMO COLLECTION BLACK JEANS RELAX S...,"97,850원",['./상품이미지/바지/데님팬츠/51006 HISHITOMO COLLECTION B...,1
87,selvedge denim pants deep indigo,"89,000원",['./상품이미지/바지/데님팬츠/selvedge denim pants deep in...,1
88,타이다이 워시드 카펜터 데님 팬츠 TP0037,"64,000원",['./상품이미지/바지/데님팬츠/타이다이 워시드 카펜터 데님 팬츠 TP00371.j...,1


In [23]:
df.columns

Index(['상품명', '가격', '이미지 경로', '이미지 개수'], dtype='object')

In [38]:
for i in range(5):
    name = df['상품명'][i]
    price = df['가격'][i]
    image_list = df['이미지 경로'][i][1:-1]
    for image in image_list.split(','):
        print(image.strip()[1:-1])

./상품이미지/바지/데님팬츠/와이드 데님 팬츠 LIGHT BLUE1.jpg
./상품이미지/바지/데님팬츠/와이드 데님 팬츠 DEEP GREY1.jpg
./상품이미지/바지/데님팬츠/1931 OBJECT JEANS CROP STRAIGHT1.jpg
./상품이미지/바지/데님팬츠/1967 JET BLACK JEANS WIDE STRAIGHT1.jpg
./상품이미지/바지/데님팬츠/8cm 숏버전 추가 0214 Moderation indigo straight fit1.jpg
./상품이미지/바지/데님팬츠/8cm 숏버전 추가 0214 Moderation indigo straight fit2.jpg
./상품이미지/바지/데님팬츠/8cm 숏버전 추가 0214 Moderation indigo straight fit3.jpg


이미지 먼저 순회하면서 저장한 후 아이템 저장하고,
아이템에 해당 이미지들과 카테고리들을 넣어줌.

In [51]:
item_save_url = 'http://127.0.0.1:8081/main/item/save'
image_save_url = 'http://127.0.0.1:8081/main/image/save'
category_to_item = 'http://127.0.0.1:8081/main/category/add-to-item'
image_to_item = 'http://127.0.0.1:8081/main/image/add-to-item'

for csv in csv_file:
    tmp = csv.split('\\')[1][:-4]
    upper, lower = tmp.split('_') # 카테고리
    df = pd.read_csv(csv)
    df = df.iloc[:, 1:]

    for i in range(90):
        name = df['상품명'][i]
        price = df['가격'][i][:-1].split(',') # '원' 제거하고 , split
        price = int(''.join(price))
        image_list = df['이미지 경로'][i][1:-1] # [, ] 제거
        images = []
        for image_path in image_list.split(','):
            images.append(image_path.strip()[1:-1]) # 공백 제거 후 ', ' 제거

        # 아이템 저장
        item_request_body = {
            "itemname":name,
            "price":price,
            "purchasecnt":"0",
            "count":"200",
            "reviewmean":"0",
            "reviewcount":"0"}
        requests.post(url=item_save_url, json=item_request_body)

        # 이미지 저장
        for img in images:
            image_request_body = {
                'imgpath':img}
            requests.post(url=image_save_url, json=image_request_body)

        # Category To Item
        for cate in [upper, lower]:
            cate_item_body = {
                "itemName":name,
                "categoryName":cate}
            requests.post(url=category_to_item, json=cate_item_body)

        # Image to Item
        for img in images:
            image_request_body = {
                "itemName":name,
                "imgPath":img}
            requests.post(url=image_to_item, json=image_request_body)